# 메일링 시스템을 위한 위험 판매자 예측 데이터 생성

## 목표
- Phase 2 모델을 활용하여 위험 판매자 예측 수행
- `risk_report_result.csv` 파일 생성 (메일링 시스템용)

## 출력 파일
- `risk_report_result.csv`: seller_id, year_month, y_pred_proba, 주요_위험사유


---
## 1. 라이브러리 임포트


In [190]:
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, 
                             accuracy_score, f1_score, roc_auc_score, 
                             precision_score, recall_score)

warnings.filterwarnings('ignore')

---
## 2. 데이터 로드 및 전처리


In [191]:
# 데이터 로드
df = pd.read_csv('../Olist_DataSet/merged_olist.csv')

In [192]:
# 필요한 컬럼만 선택
cols_needed = [
    'order_id', 'seller_id', 'order_purchase_timestamp',
    'review_score', 'has_text_review',
    'seller_delay_days', 'seller_processing_days',
    'processing_days_diff',
    'is_logistics_fault'
]

df = df[cols_needed].copy()

In [193]:
# 날짜 변환
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# 월(Year-Month) 추출
df['year_month'] = df['order_purchase_timestamp'].dt.to_period('M')

# 물류사 과실 제외
df = df[df['is_logistics_fault'] == False].copy()

print("=" * 80)
print("데이터 로드 완료 (물류사 과실 제외)")
print("=" * 80)
print(f"행 수: {len(df):,}")
print(f"판매자 수: {df['seller_id'].nunique():,}명")

데이터 로드 완료 (물류사 과실 제외)
행 수: 62,386
판매자 수: 2,635명


---
## 3. 파생 변수 생성


In [194]:
# 이진 변수 생성
df['is_processing_delayed'] = (df['processing_days_diff'] > 0).astype(int)
df['is_seller_delayed'] = (df['seller_delay_days'] > 0).astype(int)
df['is_negative_review'] = (df['review_score'] <= 3).astype(int)
df['is_critical_complaint'] = ((df['review_score'] <= 3) & (df['has_text_review'] == True)).astype(int)

---
## 4. 월별 판매자 통계 생성


In [195]:
# 월별 판매자 집계
monthly_stats = df.groupby(['seller_id', 'year_month']).agg({
    'is_processing_delayed': 'mean',   
    'is_seller_delayed': 'mean',       
    'is_negative_review': 'mean',      
    'is_critical_complaint': 'mean',   
    'seller_processing_days': ['mean', 'std'],
    'order_id': 'count'
}).reset_index()

# 컬럼명 정리
monthly_stats.columns = [
    'seller_id', 'year_month',
    'processing_delay_rate',   
    'seller_delay_rate',        
    'negative_review_rate',     
    'critical_complaint_rate',  
    'processing_days_mean', 'processing_days_std',
    'order_count'
]

# std의 NaN 처리
monthly_stats['processing_days_std'] = monthly_stats['processing_days_std'].fillna(0)

print(f"월별 통계 생성 완료: {len(monthly_stats):,}행")

월별 통계 생성 완료: 11,594행


---
## 5. 점수 계산 및 Target 생성


In [196]:
# 점수 변환 함수
def linear_score(rate_value, min_val=0.0, max_val=1.0):
    clipped = np.clip(rate_value, min_val, max_val)
    score = (clipped - min_val) / (max_val - min_val) * 100
    return score

def calculate_weighted_score_linear(processing_rate, delay_rate, negative_rate, 
                                   weight_processing=0.15, weight_delay=0.50, weight_negative=0.35):
    score_processing = linear_score(processing_rate)
    score_delay = linear_score(delay_rate)
    score_negative = linear_score(negative_rate)
    
    weighted_score = (
        score_processing * weight_processing +
        score_delay * weight_delay +
        score_negative * weight_negative
    )
    
    return weighted_score

print("점수 계산 함수 정의 완료")


점수 계산 함수 정의 완료


In [197]:
# 점수 계산
monthly_stats['weighted_score_linear_75'] = calculate_weighted_score_linear(
    monthly_stats['processing_delay_rate'],
    monthly_stats['seller_delay_rate'],
    monthly_stats['negative_review_rate']
)

# 유의판매자 여부 (50점 이상)
monthly_stats['is_seller_of_note_linear_75'] = monthly_stats['weighted_score_linear_75'] >= 50

print("점수 계산 완료")
print(f"유의 판매자: {monthly_stats['is_seller_of_note_linear_75'].sum():,}건 ({monthly_stats['is_seller_of_note_linear_75'].mean()*100:.2f}%)")


점수 계산 완료
유의 판매자: 916건 (7.90%)


---
## 6. 피처 엔지니어링


In [198]:
# 시간순 정렬
monthly_stats_enhanced = monthly_stats.sort_values(['seller_id', 'year_month']).copy()

In [199]:
# 이동 평균 피처 생성
cols_for_rolling = ['processing_delay_rate', 'seller_delay_rate', 'negative_review_rate']

for col in cols_for_rolling:
    # 2개월 이동 평균
    monthly_stats_enhanced[f'{col}_rolling_2'] = monthly_stats_enhanced.groupby('seller_id')[col].rolling(2, min_periods=1).mean().values
    # 3개월 이동 평균
    monthly_stats_enhanced[f'{col}_rolling_3'] = monthly_stats_enhanced.groupby('seller_id')[col].rolling(3, min_periods=1).mean().values

In [200]:
# 변화율 피처 생성
for col in ['processing_delay_rate', 'seller_delay_rate', 'negative_review_rate', 'order_count']:
    prev = monthly_stats_enhanced.groupby('seller_id')[col].shift(1)
    monthly_stats_enhanced[f'{col}_change'] = (monthly_stats_enhanced[col] - prev).fillna(0)

In [201]:
# 상호작용 피처 생성
monthly_stats_enhanced['delay_negative_interaction'] = monthly_stats_enhanced['seller_delay_rate'] * monthly_stats_enhanced['negative_review_rate']
monthly_stats_enhanced['processing_seller_delay_interaction'] = monthly_stats_enhanced['processing_delay_rate'] * monthly_stats_enhanced['seller_delay_rate']
monthly_stats_enhanced['total_risk_score'] = monthly_stats_enhanced['processing_delay_rate'] + monthly_stats_enhanced['seller_delay_rate'] + monthly_stats_enhanced['negative_review_rate']
monthly_stats_enhanced['avg_delay_rate'] = (monthly_stats_enhanced['processing_delay_rate'] + monthly_stats_enhanced['seller_delay_rate']) / 2

In [202]:
# 시간 피처 생성
monthly_stats_enhanced['year_month_str'] = monthly_stats_enhanced['year_month'].astype(str) + '-01'
monthly_stats_enhanced['year_month_dt'] = pd.to_datetime(monthly_stats_enhanced['year_month_str'])
monthly_stats_enhanced['month'] = monthly_stats_enhanced['year_month_dt'].dt.month
monthly_stats_enhanced['quarter'] = monthly_stats_enhanced['year_month_dt'].dt.quarter
monthly_stats_enhanced['is_month_start'] = (monthly_stats_enhanced['month'] <= 6).astype(int)
monthly_stats_enhanced['seller_tenure_months'] = monthly_stats_enhanced.groupby('seller_id').cumcount() + 1

In [203]:
print(f"\n전체 피처 개수: {len(monthly_stats_enhanced.columns)}개")


전체 피처 개수: 31개


---
## 7. Target 및 Lag 피처 생성


In [204]:
# 다음 달 Target 생성
monthly_stats_enhanced['target_is_seller_of_note_linear_75'] = monthly_stats_enhanced.groupby('seller_id')['is_seller_of_note_linear_75'].shift(-1)

In [205]:
# Lag 피처 생성 (이전 달 데이터)
enhanced_feature_cols = [
    # 기존 피처 (7개)
    'processing_delay_rate', 'seller_delay_rate', 'negative_review_rate',
    'critical_complaint_rate', 'processing_days_mean', 'processing_days_std', 'order_count',
    # 이동 평균 피처 (6개)
    'processing_delay_rate_rolling_2', 'processing_delay_rate_rolling_3',
    'seller_delay_rate_rolling_2', 'seller_delay_rate_rolling_3',
    'negative_review_rate_rolling_2', 'negative_review_rate_rolling_3',
    # 변화율 피처 (4개)
    'processing_delay_rate_change', 'seller_delay_rate_change',
    'negative_review_rate_change', 'order_count_change',
    # 상호작용 피처 (4개)
    'delay_negative_interaction', 'processing_seller_delay_interaction',
    'total_risk_score', 'avg_delay_rate',
    # 시간 피처 (4개)
    'month', 'quarter', 'is_month_start', 'seller_tenure_months'
]

for col in enhanced_feature_cols:
    if col not in ['month', 'quarter', 'is_month_start', 'seller_tenure_months']:
        monthly_stats_enhanced[f'prev_{col}'] = monthly_stats_enhanced.groupby('seller_id')[col].shift(1)
    else:
        monthly_stats_enhanced[f'prev_{col}'] = monthly_stats_enhanced[col]

print(f"Lag 피처 생성 완료: {len([col for col in enhanced_feature_cols if col not in ['month', 'quarter', 'is_month_start', 'seller_tenure_months']])}개")
monthly_stats_enhanced.head(1)


Lag 피처 생성 완료: 21개


,seller_id,year_month,processing_delay_rate,seller_delay_rate,negative_review_rate,critical_complaint_rate,processing_days_mean,processing_days_std,order_count,weighted_score_linear_75,...,prev_negative_review_rate_change,prev_order_count_change,prev_delay_negative_interaction,prev_processing_seller_delay_interaction,prev_total_risk_score,prev_avg_delay_rate,prev_month,prev_quarter,prev_is_month_start,prev_seller_tenure_months
0,001cca7ae9ae17fb1caed9dfb1094831,2017-02,0.0,0.0,0.0,0.0,0.95,0.0,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,1,1


---
## 8. 최종 데이터셋 생성


In [206]:
# 최종 피처 컬럼 정의
final_enhanced_feature_cols = [f'prev_{col}' for col in enhanced_feature_cols]

# 데이터셋 생성
final_cols_enhanced = ['seller_id', 'year_month'] + final_enhanced_feature_cols + ['target_is_seller_of_note_linear_75']

# 결측치 제거
df_final_enhanced = monthly_stats_enhanced[final_cols_enhanced].dropna().copy()

# Target을 int로 변환
df_final_enhanced['target_is_seller_of_note_linear_75'] = df_final_enhanced['target_is_seller_of_note_linear_75'].astype(int)

print("=" * 80)
print("최종 데이터셋 생성 완료")
print("=" * 80)
print(f"  - 전체 행 수: {len(monthly_stats_enhanced):,}")
print(f"  - 결측치 제거 후: {len(df_final_enhanced):,}")
print(f"  - 제거된 행: {len(monthly_stats_enhanced) - len(df_final_enhanced):,}")
print(f"  - 피처 개수: {len(final_enhanced_feature_cols)}개")

# Target 분포
target_counts = df_final_enhanced['target_is_seller_of_note_linear_75'].value_counts()
print(f"\n[Target 분포]")
print(f"  - 정상: {target_counts.get(0, 0):,}건 ({target_counts.get(0, 0)/len(df_final_enhanced)*100:.2f}%)")
print(f"  - 유의 판매자: {target_counts.get(1, 0):,}건 ({target_counts.get(1, 0)/len(df_final_enhanced)*100:.2f}%)")


최종 데이터셋 생성 완료
  - 전체 행 수: 11,594
  - 결측치 제거 후: 7,038
  - 제거된 행: 4,556
  - 피처 개수: 25개

[Target 분포]
  - 정상: 6,577건 (93.45%)
  - 유의 판매자: 461건 (6.55%)


---
## 9. Train/Test 분할


In [207]:
# 피처와 타깃 분리
X_enhanced = df_final_enhanced[final_enhanced_feature_cols]
y_enhanced = df_final_enhanced['target_is_seller_of_note_linear_75']

print(f"[데이터 준비 완료]")
print(f"  - X shape: {X_enhanced.shape}")
print(f"  - y shape: {y_enhanced.shape}")
print(f"  - 피처 수: {X_enhanced.shape[1]}개 (기존 7개 → {X_enhanced.shape[1]}개)")

# Train/Test 분할 (80:20, 시계열 순서 유지)
split_idx = int(len(X_enhanced) * 0.8)
X_train_enh = X_enhanced.iloc[:split_idx]
X_test_enh = X_enhanced.iloc[split_idx:]
y_train_enh = y_enhanced.iloc[:split_idx]
y_test_enh = y_enhanced.iloc[split_idx:]

print(f"\n[Train/Test 분할]")
print(f"  - Train: {len(X_train_enh):,}건 ({len(X_train_enh)/len(X_enhanced)*100:.1f}%)")
print(f"  - Test:  {len(X_test_enh):,}건 ({len(X_test_enh)/len(X_enhanced)*100:.1f}%)")
print(f"\n[Train Target 분포]")
print(f"  - 정상: {(y_train_enh == 0).sum():,}건 ({(y_train_enh == 0).mean()*100:.2f}%)")
print(f"  - 유의 판매자: {(y_train_enh == 1).sum():,}건 ({(y_train_enh == 1).mean()*100:.2f}%)")
print(f"\n[Test Target 분포]")
print(f"  - 정상: {(y_test_enh == 0).sum():,}건 ({(y_test_enh == 0).mean()*100:.2f}%)")
print(f"  - 유의 판매자: {(y_test_enh == 1).sum():,}건 ({(y_test_enh == 1).mean()*100:.2f}%)")

# 클래스 가중치 계산
n_samples = len(y_train_enh)
n_positive = (y_train_enh == 1).sum()
n_negative = (y_train_enh == 0).sum()
scale_pos_weight_enh = n_negative / n_positive

print(f"\n[클래스 불균형 대응]")
print(f"  - scale_pos_weight: {scale_pos_weight_enh:.2f}")

[데이터 준비 완료]
  - X shape: (7038, 25)
  - y shape: (7038,)
  - 피처 수: 25개 (기존 7개 → 25개)

[Train/Test 분할]
  - Train: 5,630건 (80.0%)
  - Test:  1,408건 (20.0%)

[Train Target 분포]
  - 정상: 5,251건 (93.27%)
  - 유의 판매자: 379건 (6.73%)

[Test Target 분포]
  - 정상: 1,326건 (94.18%)
  - 유의 판매자: 82건 (5.82%)

[클래스 불균형 대응]
  - scale_pos_weight: 13.85


---
## 11. RandomForest 모델 학습


In [208]:
# RandomForest 모델 초기화 및 학습
print("=" * 80)
print("RandomForest 모델 학습 시작")
print("=" * 80)

# RandomForest 모델 초기화 및 학습
model_rf_enh = RandomForestClassifier(
    class_weight='balanced',
    max_depth=10,  # 더 깊은 트리
    n_estimators=200,  # 더 많은 트리
    random_state=42,
    n_jobs=-1
)

model_rf_enh.fit(X_train_enh, y_train_enh)

# 예측
y_pred_rf_enh = model_rf_enh.predict(X_test_enh)
y_pred_proba_rf_enh = model_rf_enh.predict_proba(X_test_enh)[:, 1]


RandomForest 모델 학습 시작


In [209]:
# 성능 평가
acc_rf_enh = accuracy_score(y_test_enh, y_pred_rf_enh)
prec_rf_enh = precision_score(y_test_enh, y_pred_rf_enh)
rec_rf_enh = recall_score(y_test_enh, y_pred_rf_enh)
f1_rf_enh = f1_score(y_test_enh, y_pred_rf_enh)
roc_auc_rf_enh = roc_auc_score(y_test_enh, y_pred_proba_rf_enh)
cm_rf_enh = confusion_matrix(y_test_enh, y_pred_rf_enh)

print(f"\n[RandomForest 성능 (Enhanced)]")
print(f"  - Accuracy:  {acc_rf_enh:.4f}")
print(f"  - Precision: {prec_rf_enh:.4f}")
print(f"  - Recall:    {rec_rf_enh:.4f}")
print(f"  - F1-Score:  {f1_rf_enh:.4f}")
print(f"  - ROC-AUC:   {roc_auc_rf_enh:.4f}")

print(f"\n[Confusion Matrix]")
print(f"  TN={cm_rf_enh[0,0]:,}  FP={cm_rf_enh[0,1]:,}")
print(f"  FN={cm_rf_enh[1,0]:,}  TP={cm_rf_enh[1,1]:,}")

print(f"\n[Classification Report]")
print(classification_report(y_test_enh, y_pred_rf_enh, target_names=['정상', '유의 판매자']))


[RandomForest 성능 (Enhanced)]
  - Accuracy:  0.8935
  - Precision: 0.2119
  - Recall:    0.3049
  - F1-Score:  0.2500
  - ROC-AUC:   0.7335

[Confusion Matrix]
  TN=1,233  FP=93
  FN=57  TP=25

[Classification Report]
              precision    recall  f1-score   support

          정상       0.96      0.93      0.94      1326
      유의 판매자       0.21      0.30      0.25        82

    accuracy                           0.89      1408
   macro avg       0.58      0.62      0.60      1408
weighted avg       0.91      0.89      0.90      1408



In [210]:
# 8.9.15 Threshold 조정으로 Recall 개선
print("\n" + "=" * 80)
print("Threshold 조정으로 Recall 개선")
print("=" * 80)

# RandomForest (최고 성능 모델) 사용
y_pred_proba = y_pred_proba_rf_enh

# Threshold 범위 설정
thresholds = np.arange(0.05, 0.6, 0.05)
results_threshold = []

for threshold in thresholds:
    y_pred_thresh = (y_pred_proba >= threshold).astype(int)
    
    prec = precision_score(y_test_enh, y_pred_thresh, zero_division=0)
    rec = recall_score(y_test_enh, y_pred_thresh)
    f1 = f1_score(y_test_enh, y_pred_thresh, zero_division=0)
    
    results_threshold.append({
        'threshold': threshold,
        'precision': prec,
        'recall': rec,
        'f1_score': f1
    })

df_threshold = pd.DataFrame(results_threshold)

print("\n[Threshold별 성능]")
print(df_threshold.round(4).to_string(index=False))

# Recall >= 0.40 기준으로 최고 F1-Score 찾기
recall_target = 0.6
df_high_recall = df_threshold[df_threshold['recall'] >= recall_target]

if len(df_high_recall) > 0:
    best_thresh_recall = df_high_recall.loc[df_high_recall['f1_score'].idxmax()]
    print(f"\n[Recall >= {recall_target} 중 최고 F1-Score]")
    print(f"  Threshold: {best_thresh_recall['threshold']:.2f}")
    print(f"  Precision: {best_thresh_recall['precision']:.4f}")
    print(f"  Recall:    {best_thresh_recall['recall']:.4f}")
    print(f"  F1-Score:  {best_thresh_recall['f1_score']:.4f}")


Threshold 조정으로 Recall 개선

[Threshold별 성능]
 threshold  precision  recall  f1_score
      0.05     0.0676  0.9634    0.1263
      0.10     0.0741  0.9146    0.1371
      0.15     0.0863  0.9024    0.1576
      0.20     0.0947  0.8049    0.1694
      0.25     0.1115  0.7317    0.1935
      0.30     0.1165  0.5854    0.1943
      0.35     0.1277  0.5122    0.2044
      0.40     0.1422  0.3902    0.2085
      0.45     0.1657  0.3415    0.2231
      0.50     0.2119  0.3049    0.2500
      0.55     0.1975  0.1951    0.1963

[Recall >= 0.6 중 최고 F1-Score]
  Threshold: 0.25
  Precision: 0.1115
  Recall:    0.7317
  F1-Score:  0.1935


---
## 12. 예측 수행


In [211]:
# 최적 Threshold 적용 및 최종 평가

print("\n" + "=" * 80)
print("최적 Threshold 적용 및 최종 평가")
print("=" * 80)

# Recall 우선 Threshold 적용
optimal_threshold = best_thresh_recall['threshold']
y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)

# 성능 평가
cm_optimal = confusion_matrix(y_test_enh, y_pred_optimal)
print(f"\n[최적 Threshold: {optimal_threshold:.2f}]")
print(f"  Precision: {best_thresh_recall['precision']:.4f}")
print(f"  Recall:    {best_thresh_recall['recall']:.4f}")
print(f"  F1-Score:  {best_thresh_recall['f1_score']:.4f}")

print(f"\n[Confusion Matrix]")
print(f"  TN={cm_optimal[0,0]:,}  FP={cm_optimal[0,1]:,}")
print(f"  FN={cm_optimal[1,0]:,}  TP={cm_optimal[1,1]:,}")

print(f"\n[Classification Report]")
print(classification_report(y_test_enh, y_pred_optimal, target_names=['정상', '유의 판매자']))


최적 Threshold 적용 및 최종 평가

[최적 Threshold: 0.25]
  Precision: 0.1115
  Recall:    0.7317
  F1-Score:  0.1935

[Confusion Matrix]
  TN=848  FP=478
  FN=22  TP=60

[Classification Report]
              precision    recall  f1-score   support

          정상       0.97      0.64      0.77      1326
      유의 판매자       0.11      0.73      0.19        82

    accuracy                           0.64      1408
   macro avg       0.54      0.69      0.48      1408
weighted avg       0.92      0.64      0.74      1408



In [212]:
# 예측 확률 계산 (상호작용 피처 포함 모델 사용)
y_pred_proba_rf = model_rf_enh.predict_proba(X_test_enh)[:, 1]

# Threshold 0.25로 예측
y_pred_rf_025 = (y_pred_proba_rf >= 0.25).astype(int)

print("=" * 80)
print("예측 완료 (threshold=0.25, 상호작용 피처 포함)")
print("=" * 80)
print(f"  - 예측된 위험 판매자: {y_pred_rf_025.sum():,}명")
print(f"  - 위험 판매자 비율: {y_pred_rf_025.mean()*100:.2f}%")


예측 완료 (threshold=0.25, 상호작용 피처 포함)
  - 예측된 위험 판매자: 538명
  - 위험 판매자 비율: 38.21%


---
## 13. 메일링용 데이터프레임 생성


In [213]:
# Test 데이터의 인덱스 가져오기
test_indices = y_test_enh.index

# Prediction 데이터프레임 생성
prediction_df = df_final_enhanced.loc[test_indices, ['seller_id', 'year_month']].copy()
prediction_df['predicted'] = y_pred_rf_025
prediction_df['y_pred_proba'] = y_pred_proba_rf  # 컬럼명 변경 (메일링 시스템용)

# predicted=1만 필터링
risk_sellers = prediction_df[prediction_df['predicted'] == 1].copy()

print("=" * 80)
print("위험 판매자 분류 및 우선순위 부여")
print("=" * 80)
print(f"전체 위험 판매자: {len(risk_sellers):,}명\n")

# 우선순위 분류 (Priority Zone)
def assign_priority(prob):
    if prob >= 0.8:
        return 'RED'
    else:
        return 'YELLOW'

risk_sellers['priority'] = risk_sellers['y_pred_proba'].apply(assign_priority)

# 우선순위별 통계
red_zone = risk_sellers[risk_sellers['priority'] == 'RED']
yellow_zone = risk_sellers[risk_sellers['priority'] == 'YELLOW']

print(f"RED ZONE (즉시 대응 필요)")
print(f"   - 인원: {len(red_zone):,}명")
print(f"   - 기준: 위험 확률 ≥ 0.80")
print(f"   - 조치: 즉시 전화 확인 및 모니터링 강화\n")

print(f"YELLOW ZONE (관심 리스트)")
print(f"   - 인원: {len(yellow_zone):,}명")
print(f"   - 기준: 위험 확률 0.25~0.79")
print(f"   - 조치: 배송 현황 모니터링 (지연 시 Red Zone 상향)\n")

# 확률 높은 순으로 정렬
risk_sellers = risk_sellers.sort_values('y_pred_proba', ascending=False)

print(f"필터링 완료: 총 {len(risk_sellers):,}명")

위험 판매자 분류 및 우선순위 부여
전체 위험 판매자: 538명

RED ZONE (즉시 대응 필요)
   - 인원: 10명
   - 기준: 위험 확률 ≥ 0.80
   - 조치: 즉시 전화 확인 및 모니터링 강화

YELLOW ZONE (관심 리스트)
   - 인원: 528명
   - 기준: 위험 확률 0.25~0.79
   - 조치: 배송 현황 모니터링 (지연 시 Red Zone 상향)

필터링 완료: 총 538명


---
## 14. 위험사유 매핑


In [214]:
# 위험사유 매핑
risk_names = {
    'processing_delay': '처리지연율',
    'seller_delay': '출고지연율',
    'negative_review': '부정리뷰율',
    'processing_delay_trend': '처리지연율 추세',
    'seller_delay_trend': '출고지연율 추세'
}

# 각 판매자의 위험사유 찾기
risk_reasons = []

for idx in risk_sellers.index:
    features = X_test_enh.loc[idx] 
    
    # 위험 관련 피처만 가져오기
    processing_delay = features['prev_processing_delay_rate']
    seller_delay = features['prev_seller_delay_rate']
    negative_review = features['prev_negative_review_rate']
    processing_delay_change = features['prev_processing_delay_rate_change']
    seller_delay_change = features['prev_seller_delay_rate_change']
    
    reasons = []
    if processing_delay >= 0.5:
        reasons.append(f"{risk_names['processing_delay']} 높음({processing_delay*100:.0f}%)")
    if seller_delay >= 0.5:
        reasons.append(f"{risk_names['seller_delay']} 높음({seller_delay*100:.0f}%)")
    if negative_review >= 0.3:
        reasons.append(f"{risk_names['negative_review']} 높음({negative_review*100:.0f}%)")
    if processing_delay_change > 0.2:
        reasons.append(f"{risk_names['processing_delay_trend']} 높음({processing_delay_change*100:.0f}%)")
    if seller_delay_change > 0.2:
        reasons.append(f"{risk_names['seller_delay_trend']} 높음({seller_delay_change*100:.0f}%)")
    
    risk_reasons.append(" | ".join(reasons) if reasons else "지연/리뷰 문제")

risk_sellers['주요_위험사유'] = risk_reasons

---
## 15. CSV 저장 (메일링 시스템용)


In [215]:
# 필요한 컬럼만 선택 (priority 포함)
risk_sellers = risk_sellers[['seller_id', 'year_month', 'y_pred_proba', 'priority', '주요_위험사유']]

# CSV 저장 (메일링 시스템에서 읽을 파일)
output_path = 'risk_report_result.csv'
risk_sellers.to_csv(output_path, index=False, encoding='utf-8-sig')

print("=" * 80)
print("메일링용 데이터 생성 완료")
print("=" * 80)
print(f"저장 경로: {output_path}")
print(f"총 위험 판매자: {len(risk_sellers):,}명")
print(f" RED ZONE: {(risk_sellers['priority'] == 'RED').sum():,}명")
print(f" YELLOW ZONE: {(risk_sellers['priority'] == 'YELLOW').sum():,}명")

print(f"\n[RED ZONE Top 10 - 즉시 대응 필요]")
red_top10 = risk_sellers[risk_sellers['priority'] == 'RED'].head(10)
if len(red_top10) > 0:
    print(red_top10[['seller_id', 'year_month', 'y_pred_proba', '주요_위험사유']].to_string(index=False))
else:
    print("RED ZONE 판매자 없음")

print(f"\n[전체 위험 판매자 Top 10]")
print(risk_sellers.head(10).to_string(index=False))


메일링용 데이터 생성 완료
저장 경로: risk_report_result.csv
총 위험 판매자: 538명
 RED ZONE: 10명
 YELLOW ZONE: 528명

[RED ZONE Top 10 - 즉시 대응 필요]
                       seller_id year_month  y_pred_proba                                             주요_위험사유
d93844a9c55ba7ce353388bcf849ea56    2017-05      0.871529                     처리지연율 높음(100%) | 출고지연율 높음(100%)
f5f46307a4d15880ca14fab4ad9dfc9b    2017-08      0.848681                     처리지연율 높음(100%) | 출고지연율 높음(100%)
efcd8d2104f1a05d028af7bad20d974b    2017-06      0.846970                     처리지연율 높음(100%) | 출고지연율 높음(100%)
f1b93673502375d491780bb49d615dbc    2018-06      0.843566                     처리지연율 높음(100%) | 출고지연율 높음(100%)
d93844a9c55ba7ce353388bcf849ea56    2017-04      0.843502                     처리지연율 높음(100%) | 출고지연율 높음(100%)
df0f42bc4c2142eacf0eaf2cffd0cfbb    2018-02      0.840152                     처리지연율 높음(100%) | 출고지연율 높음(100%)
f25e239052084705e17a982bc600ab2a    2018-03      0.821946 처리지연율 높음(100%) | 출고지연율 높음(100%) | 출고지연율 추세 높음(10